## Modeling

With the data successfully obtained and wrangled, we can finally put it into some models!

In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [60]:
df = pd.read_csv('training-data.csv')

print(df.shape)
df.head()

(18329, 104)


,Player,Lge,Tm,Year,Pos,Age,G,GS,MP,FG,...,TM_TRB_adj,TM_STL_adj,TM_BLK_adj,TM_TOV_adj,TM_PF_adj,TM_PTS_adj,TM_AST_adj,TM_CAS_adj,Sels,Target
0,A.C. Green,NBA,LAL,1986.0,4.0,22.0,82.0,1.0,18.8,2.5,...,113.353333,20.257500,13.859167,47.038333,52.697500,291.443333,84.985833,75.868561,0.0,1.1
1,A.C. Green,NBA,LAL,1987.0,4.0,23.0,79.0,72.0,28.4,4.0,...,117.330000,24.980000,14.811818,53.943636,64.157273,346.999091,103.644545,93.370094,0.0,1.1
2,A.C. Green,NBA,LAL,1988.0,4.0,24.0,82.0,64.0,32.1,3.9,...,112.674167,22.459167,14.959167,42.338333,52.324167,314.798333,81.284167,81.034171,0.0,1.3
3,A.C. Green,NBA,LAL,1989.0,4.0,25.0,82.0,82.0,30.6,4.9,...,138.156364,27.462727,15.890909,54.841818,68.721818,396.150000,101.753636,93.719806,0.0,1.1
4,A.C. Green,NBA,LAL,1990.0,4.0,26.0,82.0,82.0,33.0,4.7,...,146.236667,28.225556,13.398889,53.021111,59.227778,389.544444,100.957778,101.027904,1.0,0.9


In [61]:
df['Year'].describe() # The year range is accurate--recall that I don't want to touch the 2019 data until the very end.

count    18329.000000
mean      1991.977686
std         17.942735
min       1947.000000
25%       1979.000000
50%       1995.000000
75%       2007.000000
max       2018.000000
Name: Year, dtype: float64

In [62]:
# I'll take a quick look at my categorical data.

df.describe(exclude=np.number)

,Player,Lge,Tm
count,18329,18329,18329
unique,3308,3,103
top,Mike Dunleavy,NBA,NYK
freq,25,17321,826


Since the goal is to predict each player's assists for 2019-2020, it makes the most sense to me to do a train/test split on "Year," with the test being the latest year in the training data (i.e. 2018).

To avoid over-fitting to the test set, I'll also make a val set. I tried various train/val splits, and putting it at around 2014 ended up being for the best.

In [63]:
test = df[df['Year'] == 2018]
temp = df[df['Year'] < 2018]
train = temp[temp['Year'] < 2014]
val = temp[temp['Year'] >= 2014]

### Calculating the baseline

While determining the baseline prediction can be somewhat subjective, in this case I think it makes the most sense to have the baseline be the *previous* year's assists total. In other words, if I want to guess how many assists a player will have this year, a good first pass is to say that they'll have just as many as they did last year.

Thankfully, the Assist column already provides us with this information.

In [64]:
from sklearn.metrics import mean_squared_error

baseline = val['AST']

baseline_mse = mean_squared_error(baseline, val['Target'])

baseline_mse

# The baseline's mean squared error is quite low, so it might be difficult for my models to beat it, at least by much.

0.8528051787916153

### Linear Models

In [65]:
# One-hot encoding is the best way to deal with categorical variables in linear models.
# "PLayer" and "Team" have high cardinality, and I don't think either is very meaningful, so I'll just
# remove them beforehand. I also have to drop "Target" from the input matrix, obviously.

non_features = ['Player', 'Tm', 'Target']
X_train = train.drop(non_features, axis=1)
y_train = train['Target']
X_val = val.drop(non_features, axis=1)
y_val = val['Target']

# After determining my hyperparameters, I'll want to use all the data I have available to predict the test data.
# So along with the above, I'm also going to make "X_train_full" dataframes with everything before 2018 included.
X_train_full = temp.drop(non_features, axis=1)
y_train_full = temp['Target']
X_test = test.drop(non_features, axis=1)
y_test = test['Target']

In [66]:
import category_encoders as ce

encoder = ce.OneHotEncoder(use_cat_names=True)

X_train_encoded = encoder.fit_transform(X_train)
X_val_encoded = encoder.transform(X_val)

X_train_full_encoded = encoder.fit_transform(X_train_full)
X_test_encoded = encoder.transform(X_test)

In [67]:
# First is a basic linear regression. I'll also use Select K Best to determine the optimal number of features.

from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.linear_model import LinearRegression

lowest = [0.9, 1] # This will help me keep track of which k value is best.
for k in range(1, len(X_train_encoded.columns)+1):
    print(f'{k} features')
    
    selector = SelectKBest(score_func=f_regression, k=k)
    X_train_selected = selector.fit_transform(X_train_encoded, y_train)
    X_val_selected = selector.transform(X_val_encoded)
    
    model = LinearRegression()
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_val_selected)
    
    mse = mean_squared_error(y_val, y_pred)
    
    if mse < lowest[0]:
        lowest[0] = mse
        lowest[1] = k
    
    print(f'Val MSE: {mse} \n')
    print(f'Lowest so far: k = {lowest[1]} with mse = {lowest[0]}' )

1 features
Val MSE: 0.7850083241821225 

Lowest so far: k = 1 with mse = 0.7850083241821225
2 features
Val MSE: 0.7860222575464023 

Lowest so far: k = 1 with mse = 0.7850083241821225
3 features
Val MSE: 0.7771730114575385 

Lowest so far: k = 3 with mse = 0.7771730114575385
4 features
Val MSE: 0.7744031029516313 

Lowest so far: k = 4 with mse = 0.7744031029516313
5 features
Val MSE: 0.7705570833891177 

Lowest so far: k = 5 with mse = 0.7705570833891177
6 features
Val MSE: 0.7678744871431867 

Lowest so far: k = 6 with mse = 0.7678744871431867
7 features
Val MSE: 0.7682631176818985 

Lowest so far: k = 6 with mse = 0.7678744871431867
8 features
Val MSE: 0.7642486297443156 

Lowest so far: k = 8 with mse = 0.7642486297443156
9 features
Val MSE: 0.7642676598713812 

Lowest so far: k = 8 with mse = 0.7642486297443156
10 features
Val MSE: 0.7583718820413511 

Lowest so far: k = 10 with mse = 0.7583718820413511
11 features
Val MSE: 0.7525625297481134 

Lowest so far: k = 11 with mse = 0.7

In [68]:
# Looks like k=100 is best.

selector = SelectKBest(score_func=f_regression, k=100)

X_train_full_selected = selector.fit_transform(X_train_full_encoded, y_train_full)
X_test_selected = selector.transform(X_test_encoded)

model = LinearRegression()

model.fit(X_train_full_selected, y_train_full)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [69]:
# I'm going to be testing a lot of models, so let's make the next part a function.

def determine_error(test, model):

    y_pred = model.predict(test)

    model_mse = mean_squared_error(y_test, y_pred)

    improvement = baseline_mse - model_mse

    percent_improve = (improvement / baseline_mse) * 100

    print('Model mean squared error:', model_mse)
    print('Baseline mean squared error:', baseline_mse)
    print('Improvement over baseline:', improvement)
    print(f'Percent improvement: {percent_improve}%')
    
determine_error(test=X_test_selected, model=model)

# Nine percent is actually pretty good, considering how accurate the baseline was. But let's see if we can do better!

Model mean squared error: 0.7747338701375271
Baseline mean squared error: 0.8528051787916153
Improvement over baseline: 0.07807130865408818
Percent improvement: 9.154647579029895%


In [70]:
# Let's do a Ridge regression next. I'll do a similar for loop to get the optimal alpha value.

from sklearn.linear_model import Ridge

lowest = [1, 0]
for alpha in range(0, 1000, 1):
    ridge_reg_split = Ridge(alpha=alpha).fit(X_train_encoded, y_train)
    mse = mean_squared_error(y_val, ridge_reg_split.predict(X_val_encoded))
    if mse < lowest[0]:
        lowest[0] = mse
        lowest[1] = alpha
    print(f'For alpha = {alpha}, mse = {mse}')
    print(f'Lowest so far: alpha = {lowest[1]}, mse = {lowest[0]}')

For alpha = 0, mse = 0.7073693908932405
Lowest so far: alpha = 0, mse = 0.7073693908932405
For alpha = 1, mse = 0.705697483393456
Lowest so far: alpha = 1, mse = 0.705697483393456
For alpha = 2, mse = 0.7054815483608411
Lowest so far: alpha = 2, mse = 0.7054815483608411
For alpha = 3, mse = 0.7053426140151803
Lowest so far: alpha = 3, mse = 0.7053426140151803
For alpha = 4, mse = 0.7052368242058376
Lowest so far: alpha = 4, mse = 0.7052368242058376


C:\Anaconda3\envs\unit2\lib\site-packages\sklearn\linear_model\ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=1.54277e-22): result may not be accurate.
  overwrite_a=True).T


For alpha = 5, mse = 0.7051503268401473
Lowest so far: alpha = 5, mse = 0.7051503268401473
For alpha = 6, mse = 0.7050767741903321
Lowest so far: alpha = 6, mse = 0.7050767741903321
For alpha = 7, mse = 0.705012632766067
Lowest so far: alpha = 7, mse = 0.705012632766067
For alpha = 8, mse = 0.7049556882583278
Lowest so far: alpha = 8, mse = 0.7049556882583278
For alpha = 9, mse = 0.7049044409924862
Lowest so far: alpha = 9, mse = 0.7049044409924862
For alpha = 10, mse = 0.7048578191635547
Lowest so far: alpha = 10, mse = 0.7048578191635547
For alpha = 11, mse = 0.7048150262412208
Lowest so far: alpha = 11, mse = 0.7048150262412208
For alpha = 12, mse = 0.7047754524961817
Lowest so far: alpha = 12, mse = 0.7047754524961817
For alpha = 13, mse = 0.7047386202279142
Lowest so far: alpha = 13, mse = 0.7047386202279142
For alpha = 14, mse = 0.7047041480787114
Lowest so far: alpha = 14, mse = 0.7047041480787114
For alpha = 15, mse = 0.7046717268206953
Lowest so far: alpha = 15, mse = 0.704671

In [13]:
# Let's go with alpha=504, then!

model = Ridge(alpha=504)

model.fit(X_train_full_encoded, y_train_full)

determine_error(test=X_test_encoded, model=model)

# Just a little better than the basic linear regression.

Model mean squared error: 0.7727204860069009
Baseline mean squared error: 0.8528051787916153
Improvement over baseline: 0.08008469278471442
Percent improvement: 9.390737155019467%


### Random forest models

In [14]:
# Random forests do fine with ordinal encoding, so we can include the Team column this time.

non_features = ['Player', 'Target']
X_train = train.drop(non_features, axis=1)
y_train = train['Target']
X_val = val.drop(non_features, axis=1)
y_val = val['Target']

X_train_full = temp.drop(non_features, axis=1)
y_train_full = temp['Target']
X_test = test.drop(non_features, axis=1)
y_test = test['Target']

In [15]:
# I'll do cross-validation to determine the optimal hyperparameters.

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from scipy.stats import randint, uniform

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    RandomForestRegressor(random_state=100)
)

param_distributions = {
    'randomforestregressor__n_estimators': randint(50, 500),
    'randomforestregressor__max_depth': [5, 10, 15, 20, None],
    'randomforestregressor__max_features': uniform(0,1),
}

search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=20,
    cv=5,
    scoring='neg_mean_squared_error',
    verbose=10,
    return_train_score=True,
    n_jobs=-1
)

search.fit(X_train_full, y_train_full);

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   35.4s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:  9.8min
[Parallel(n_jobs=-1)]: Done  61 tasks      | elapsed: 11.6min
[Parallel(n_jobs=-1)]: Done  74 tasks      | elapsed: 12.1min
[Parallel(n_jobs=-1)]: Done  88 out of 100 | elapsed: 18.7min remaining:  2.6min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 23.3min finished


In [16]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation mse', -search.best_score_)

Best hyperparameters {'randomforestregressor__max_depth': 10, 'randomforestregressor__max_features': 0.4033123004286774, 'randomforestregressor__n_estimators': 343}
Cross-validation mse 0.8044576974020399


In [17]:
param_values = list(search.best_params_.values())

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    RandomForestRegressor(max_depth=param_values[0], max_features=param_values[1],
                          n_estimators=param_values[2], random_state=100)
)

pipeline.fit(X_train_full, y_train_full)

determine_error(test=X_test, model=pipeline)

# Worse than both linear models.

Model mean squared error: 0.7933018091264973
Baseline mean squared error: 0.8528051787916153
Improvement over baseline: 0.059503369665118044
Percent improvement: 6.977369643724668%


In [33]:
# And finally, XGBoost.
# Since it doesn't play great with pipelines, I'll encode in advance.

encoder2 = ce.OrdinalEncoder()
X_train_encoded2 = encoder2.fit_transform(X_train)
X_val_encoded2 = encoder2.transform(X_val)

from xgboost import XGBRegressor

eval_set = [(X_train_encoded2, y_train),
            (X_val_encoded2, y_val)]

model = XGBRegressor(
    n_estimators=1000,
    max_depth=3,
    learning_rate=0.1, # I tested various combinations and this seemed to be the best.
    n_jobs=-1
)

model.fit(X_train_encoded2, y_train, eval_set=eval_set, eval_metric='rmse', early_stopping_rounds=50)

[05:17:01] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:2.30546	validation_1-rmse:2.14346
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 50 rounds.
[1]	validation_0-rmse:2.11773	validation_1-rmse:1.9674
[2]	validation_0-rmse:1.95206	validation_1-rmse:1.81148
[3]	validation_0-rmse:1.8059	validation_1-rmse:1.67545
[4]	validation_0-rmse:1.67815	validation_1-rmse:1.55746
[5]	validation_0-rmse:1.5664	validation_1-rmse:1.45121
[6]	validation_0-rmse:1.46909	validation_1-rmse:1.36233
[7]	validation_0-rmse:1.38453	validation_1-rmse:1.288
[8]	validation_0-rmse:1.31137	validation_1-rmse:1.22297
[9]	validation_0-rmse:1.24871	validation_1-rmse:1.16494
[10]	validation_0-rmse:1.19483	validation_1-rmse:1.11798
[11]	validation_0-rmse:1.14891	validation_1-rmse:1.07578
[12]	validation_0-rmse:1.109	validation_1-rmse:1.04149
[

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=-1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [34]:
# Now to do it on my test set.

X_train_full_encoded2 = encoder2.fit_transform(X_train_full)
X_test_encoded2 = encoder2.transform(X_test)

eval_set = [(X_train_full_encoded2, y_train_full),
            (X_test_encoded2, y_test)]

model = XGBRegressor(
    n_estimators=1000,
    max_depth=3,
    learning_rate=0.1,
    n_jobs=-1
)

model.fit(X_train_encoded2, y_train, eval_set=eval_set, eval_metric='rmse', early_stopping_rounds=50)

[05:17:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:2.29126	validation_1-rmse:2.2893
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 50 rounds.
[1]	validation_0-rmse:2.10457	validation_1-rmse:2.10787
[2]	validation_0-rmse:1.93975	validation_1-rmse:1.94958
[3]	validation_0-rmse:1.79448	validation_1-rmse:1.80953
[4]	validation_0-rmse:1.66758	validation_1-rmse:1.68824
[5]	validation_0-rmse:1.55632	validation_1-rmse:1.58081
[6]	validation_0-rmse:1.45974	validation_1-rmse:1.48811
[7]	validation_0-rmse:1.37607	validation_1-rmse:1.4098
[8]	validation_0-rmse:1.30361	validation_1-rmse:1.33964
[9]	validation_0-rmse:1.24136	validation_1-rmse:1.27899
[10]	validation_0-rmse:1.18808	validation_1-rmse:1.22948
[11]	validation_0-rmse:1.14248	validation_1-rmse:1.18366
[12]	validation_0-rmse:1.10305	validation_1-rmse:1.14

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=-1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [35]:
determine_error(test=X_test_encoded2, model=model)

# Better than the random forests, but still a bit worse than the linear models.

Model mean squared error: 0.7794797007022337
Baseline mean squared error: 0.8528051787916153
Improvement over baseline: 0.07332547808938161
Percent improvement: 8.598151126765007%


### Final model

So it's been decided: my model is a Ridge Regression with alpha=504.

Now the final step is to bring all my data together, put it into the model, and make predictions for the 2019-2020 season!

In [71]:
df2 = pd.read_csv('wrangled-data.csv')

test_final = df2[df2['Year'] == 2019]

In [72]:
non_features = ['Player', 'Tm', 'Target']
X_train_final = df.drop(non_features, axis=1)
y_train_final = df['Target']

X_test_final = test_final.drop(non_features, axis=1)

In [73]:
encoder = ce.OneHotEncoder(use_cat_names=True)

X_train_final_encoded = encoder.fit_transform(X_train_final)
X_test_final_encoded = encoder.transform(X_test_final)

In [74]:
model = Ridge(alpha=504)

model.fit(X_train_final_encoded, y_train_final)

Ridge(alpha=504, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [75]:
y_pred = model.predict(X_test_final_encoded)

In [76]:
test_final['preds'] = y_pred

test_final = test_final.drop('Target', axis=1)

test_final.head()

C:\Anaconda3\envs\unit2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Player,Lge,Tm,Year,Pos,Age,G,GS,MP,FG,...,TM_TRB_adj,TM_STL_adj,TM_BLK_adj,TM_TOV_adj,TM_PF_adj,TM_PTS_adj,TM_AST_adj,TM_CAS_adj,Sels,preds
45,Aaron Gordon,NBA,ORL,2019.0,4.0,23.0,78.0,78.0,33.8,6.0,...,102.258333,19.840000,11.965000,33.768333,48.170833,264.368333,56.255833,46.034645,0.0,3.199613
56,Aaron Holiday,NBA,IND,2019.0,1.0,22.0,50.0,0.0,12.9,2.1,...,98.770667,23.122000,10.997333,33.052667,45.211333,281.136667,60.038667,51.608909,0.0,2.164918
93,Abdel Nader,NBA,OKC,2019.0,3.0,25.0,61.0,1.0,11.4,1.5,...,103.825000,22.044286,9.081429,39.164286,47.000714,272.657143,65.767857,59.281408,0.0,0.723526
258,Al Horford,NBA,BOS,2019.0,5.0,32.0,68.0,68.0,29.0,5.7,...,92.983333,15.905333,9.110000,29.272000,39.791333,235.256000,50.416000,43.692085,0.0,3.189894
318,Al-Farouq Aminu,NBA,POR,2019.0,4.0,28.0,81.0,81.0,28.3,3.2,...,99.217333,14.686667,10.374667,30.038000,40.424667,262.468000,44.823333,43.523000,0.0,1.229861


In [77]:
test_final.to_csv('final-predictions.csv', index=None, header=True)